# Module 12 Challenge
## Deliverable 2: Scrape and Analyze Mars Weather Data

In [ ]:
# Import relevant libraries
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

### Step 1: Visit the Website

Use automated browsing to visit the [Mars Temperature Data Site](https://data-class-mars-challenge.s3.amazonaws.com/Mars/index.html). Inspect the page to identify which elements to scrape. Note that the URL is `https://data-class-mars-challenge.s3.amazonaws.com/Mars/index.html`.

   > **Hint** To identify which elements to scrape, you might want to inspect the page by using Chrome DevTools to discover whether the table contains usable classes.


In [ ]:
# Visit the website
# https://data-class-mars-challenge.s3.amazonaws.com/Mars/index.html

url = "https://data-class-mars-challenge.s3.amazonaws.com/Mars/index.html"
browser.visit(url)

### Step 2: Scrape the Table

Create a Beautiful Soup object and use it to scrape the data in the HTML table.

Note that this can also be achieved by using the Pandas `read_html` function. However, use Beautiful Soup here to continue sharpening your web scraping skills.

In [ ]:
# Create a Beautiful Soup Object
html = browser.html
soup = soup(html, 'html.parser')

In [ ]:
# Extract all rows of data
results = soup.find('table', class_='table')

### Step 3: Store the Data

Assemble the scraped data into a Pandas DataFrame. The columns should have the same headings as the table on the website. Here’s an explanation of the column headings:

* `id`: the identification number of a single transmission from the Curiosity rover
* `terrestrial_date`: the date on Earth
* `sol`: the number of elapsed sols (Martian days) since Curiosity landed on Mars
* `ls`: the solar longitude
* `month`: the Martian month
* `min_temp`: the minimum temperature, in Celsius, of a single Martian day (sol)
* `pressure`: The atmospheric pressure at Curiosity's location

In [ ]:
headers = soup.find_all('th')
info = soup.find_all('tr', class_='data-row')

In [ ]:
# Create an empty list

table_headers = []
id = []
terrestrial_date = []
sol = []
ls = []
month = []
min_temp = []
pressure = []


# retrieving data
for i in info:
   # appending id
    mars_id = i.find('td')
    id.append(int(mars_id.text))
    
    # appending terrestial dates
    terrestrial_date.append(mars_id.find_next_sibling('td').text)
    mars_id = mars_id.find_next_sibling('td')
    
    # appending sol
    sol.append(int(mars_id.find_next_sibling("td").text))
    mars_id = mars_id.find_next_sibling("td")
    
    # appending ls
    ls.append(int(mars_id.find_next_sibling("td").text))
    mars_id=mars_id.find_next_sibling('td')
    
    # appending month
    month.append(int(mars_id.find_next_sibling("td").text))
    mars_id = mars_id.find_next_sibling("td")
    
    # appending min_temp
    min_temp.append(float(mars_id.find_next_sibling("td").text))
    mars_id = mars_id.find_next_sibling("td")
    
    # appending pressure
    pressure.append(float(mars_id.find_next_sibling("td").text))

In [ ]:
# Create a Pandas DataFrame by using the list of rows and a list of the column names
df=pd.DataFrame({'id': id,  'terrestrial_date': terrestrial_date, 'sol': sol,
                        'ls': ls, 'month': month, 'min_temp': min_temp, 'pressure':pressure})

In [ ]:
# Confirm DataFrame was created successfully
df

### Step 4: Prepare Data for Analysis

Examine the data types that are currently associated with each column. If necessary, cast (or convert) the data to the appropriate `datetime`, `int`, or `float` data types.

  > **Hint** You can use the Pandas `astype` and `to_datetime` methods to accomplish this task.


In [ ]:
# Examine data type of each column
examine=df.dtypes
examine

In [ ]:
# Change data types for data analysis
df = df.astype({'terrestrial_date':'datetime64','sol':'int64', 'ls':'int64', 'month':'int64', 'min_temp':'float64',
               'pressure':'float64'})

In [ ]:
# Confirm type changes were successful by examining data types again
examine=df.dtypes
examine

### Step 5: Analyze the Data

Analyze your dataset by using Pandas functions to answer the following questions:

1. How many months exist on Mars?
2. How many Martian (and not Earth) days worth of data exist in the scraped dataset?
3. What are the coldest and the warmest months on Mars (at the location of Curiosity)? To answer this question:
    * Find the average the minimum daily temperature for all of the months.
    * Plot the results as a bar chart.
4. Which months have the lowest and the highest atmospheric pressure on Mars? To answer this question:
    * Find the average the daily atmospheric pressure of all the months.
    * Plot the results as a bar chart.
5. About how many terrestrial (Earth) days exist in a Martian year? To answer this question:
    * Consider how many days elapse on Earth in the time that Mars circles the Sun once.
    * Visually estimate the result by plotting the daily minimum temperature.


In [ ]:
# 1. How many months are there on Mars?
count = df["month"].nunique()
print(f'There are {count} months on Mars.')

In [ ]:
# 2. How many Martian days' worth of data are there?
days_on_mars = df["sol"].max() - df["sol"].min() + 1
print(f'There are {days_on_mars} days on Mars in current dataset.')

In [ ]:
# 3. What is the average low temperature by month?
average_temp = df.groupby("month")
average_temp["min_temp"].mean()

In [ ]:
# Plot the average temperature by month
plot1 = average_temp["min_temp"].mean()
plot1.plot.bar()
plt.ylabel("Temperature in Celsius")
plt.show()

In [ ]:
# Identify the coldest and hottest months in Curiosity's location
plot1_sorted = plot1.sort_values(ascending=True)
plot1_sorted.plot.bar()
plt.ylabel("Temperature in Celsius")
plt.show()

In [ ]:
# 4. Average pressure by Martian month
average_pressure = df.groupby("month")
average_pressure["pressure"].mean()

In [ ]:
# Plot the average pressure by month
plot2 = average_pressure["pressure"].mean()
plot2 = plot2.sort_values(ascending=True)
plot2.plot.bar()
plt.ylabel("Average pressure on Mars")
plt.show()


In [ ]:
# 5. How many terrestrial (earth) days are there in a Martian year?
earth_days = df["min_temp"]

earth_days.plot()
plt.xlabel("NO of days")
plt.ylabel("Minimum temp")
plt.show()
sol_days=df["sol"].count()
sol_days
print(f'Graph abve present {sol_days} martian days, the distance from peak to peak is about 670 days which is supported by other google search from which we know thatthe length of one Martian year at 687 days.')

### Step 6: Save the Data

Export the DataFrame to a CSV file.

In [ ]:
# Write the data to a CSV
output_csv = 'Output/mars_weather_output_file.csv'

df.to_csv(output_csv, header=True, index=False)

In [ ]:
browser.quit()